# Prepare & Publish FC as HFS

This notebook prepares and publishes a feature class to ArcGIS Online as a hosted feature service. It ZIPs a file geodatabase containing a single feature class, uploads the zipped file to AGOL, and publishes it as a hosted feature layer. Once published, the hosted feature layer properties are updated.

TODO: Don't forget to update the path to the feature class, FGDB item name, and published service properties before running.

### Imports

In [1]:
import arcpy
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
import zipfile
import os

### Set Environment & Set Constants

In [2]:
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

#TODO: Update path to feature class
#
IN_FEATURE_CLASS = r"C:\Users\mark9020\Documents\ArcGIS\Projects\Ag Census\AG_Census_Published.gdb" \
                    r"\USDA_Census_of_Agriculture_2022"

STAGING_FGDB = r"C:\Users\mark9020\Downloads\Staging_for_Upload.gdb"
STAGING_NAME = "USDA_Census_of_Agriculture_2022"

# TODO: Update name of zipped file to refelct the next sequential release
#
ZIP_PATH = r"C:\Users\mark9020\Downloads"
ZIP_FGDB_NAME = "USDA Census of Agriculture 2022 2025R1"

### Main

In [3]:
# Copy the feature class to the staging location
#
staging_fc = os.path.join(STAGING_FGDB, STAGING_NAME)

if not arcpy.Exists(STAGING_FGDB):
    arcpy.management.CreateFileGDB(os.path.dirname(STAGING_FGDB), os.path.basename(STAGING_FGDB))
    
export_fc = arcpy.conversion.ExportFeatures(IN_FEATURE_CLASS, staging_fc)[0]

# As of Pro 3.3, adding the clear cache function to address a schema lock not being
# released after the create FGDB. Appears to be an issue previously resolved and reintroduced
# since I have found several references to this error at Esri Community
# https://community.esri.com/t5/python-questions/upgraded-to-arcgis-pro-2-6-arcpy-createfilegdb/m-p/189915
# and BUG-000133976
#
arcpy.management.ClearWorkspaceCache()

print("Feature class copied to staging FGDB.")

Feature class copied to staging FGDB.


In [4]:
# Zip up the file geodatabase
#

# zip_name = STAGING_FGDB[:-4] + ".zip"
zip_name = os.path.join(ZIP_PATH, ZIP_FGDB_NAME + ".zip")

with zipfile.ZipFile(zip_name, 'w') as zip_ref:
    for folder_name, subfolders, filenames in os.walk(STAGING_FGDB):
        for filename in filenames:
            file_path = os.path.join(folder_name, filename)
            zip_ref.write(file_path, os.path.relpath(file_path, r"C:\Users\mark9020\Downloads"))

zip_ref.close()
print("FGDB zipped.")

FGDB zipped.


In [5]:
# Log into ArcGIS Online
# This command uses a previously saved local profile to connect to the GIS.
# See https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/ for additional details
#
# Create a profile so I don't have to enter credentials after the first time
#
# dest_org = GIS("https://arcgis.com", username="esri_environment", password= "", profile="esri_environment")
# print("Successfully logged in as: " + dest_org.properties.user.username)

dest_org = GIS(r"https://arcgis.com", profile="esri_environment")
print("Successfully logged in as: " + dest_org.properties.user.username)

Successfully logged in as: esri_environment


In [6]:
%%time
# Must be the first command in the cell

# Define File Geodatabase/Feature Layer Properties
#
# TODO: Update FGDB item name
fgdb_new_properties = {
    'type': 'File Geodatabase',
    'title': ZIP_FGDB_NAME,    
}

# Get the destination folder for the zipped file geodatabase
census_folder = dest_org.content.folders.get("AG Census")

# Upload zipped file geodatabase to a folder in AGOL
#
add_job = census_folder.add(item_properties=fgdb_new_properties, file=zip_name)

if not add_job.done():
    print("...job precessing...")
else:
    new_uploaded_item = add_job.result()

print("File uploaded")

File uploaded
CPU times: total: 891 ms
Wall time: 40.9 s


In [7]:
%%time

# Publish the hosted feature service from the file geodatabase
#
published_fl = new_uploaded_item.publish()
print("Service published.")
# published_fl

Service published.
CPU times: total: 484 ms
Wall time: 2min 29s


In [8]:
# Update the properties of the new service.
#
# TODO: Update published service properties
update_prop_dict= {
    'title': ZIP_FGDB_NAME + "_DO NOT DELETE",    
    'snippet': 'This layer contains county polygons used in reporting ' \
    'the 2022 Census of Agriculture from the U.S. Department of Agriculture National ' \
    'Agriculture Statistics Service. The features are United States County Boundaries circa 2022 including total population.',
    'description': '<div><b><font color="#ff0000">Important</font></b>: This hosted feature layer supports the following ' \
    'layer views, DO NOT DELETE!<br /></div><div><br /></div>' \
    '<ul><li>USDA Census of Agriculture 2022 - Almond Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Animal Totals<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Barley Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Cattle Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Chicken Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Corn Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Cotton Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Crop Totals<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Farm Operations<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Government Programs<br /></li> ' \
    '<li>USDA Census of Agriculture 2022 - Grain Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Grape Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Hay Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Hog Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Labor<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Machinery Totals<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Milk Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Producers<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Rice Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Sorghum Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Soybean Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Tractors<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Trucks<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Turkey Production<br /></li>' \
    '<li>USDA Census of Agriculture 2022 - Wheat Production</li></ul>',
    
    'licenseInfo': 'This item is <b>not shared</b> but <b>has dependent layers (views) that are shared with the ' \
    'public</b> and may be part of the Living Atlas.',
    
    'tags': 'agriculture, census, county, USDA, 2022, animal totals, livestock, poltury, crop totals, field crops, ' \
    'expenses, income, fruit, tree nuts, farm, land, assets, farm operations, producers',
    
    'accessInformation': 'Esri, US Census Bureau, US Department of Agriculture'
}

published_fl.update(
    item_properties=update_prop_dict,
    thumbnail=r'C:\Users\mark9020\OneDrive - Esri\Pictures\LAW Thumbnails\supportiveservice.png'
)

protect_res = published_fl.protect(enable=True)
print(f"Delete protection: {protect_res['success']}")

Delete protection: True
